In [ ]:
# YOLOv8-OCHD: Wood Surface Defect Detection

This notebook prepares the dataset used for training and evaluation of:
- YOLOv8n (baseline)
- O
- OC
- OCH
- OCHD (DyHead)

Dataset:
- Source: iluvvatar/wood_surface_defects (Hugging Face)
- Total images: 20,276
- Classes: 10 wood surface defect categories

This notebook:
1. Loads the dataset
2. Converts annotations to YOLO format
3. Splits data into train/validation
4. Visualizes bounding boxes


In [1]:
import os
import random
import cv2
import torch
from datasets import load_dataset
from tqdm import tqdm

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0))

C:\YOLOv8_OCHD_Project\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Torch version: 2.9.1+cu126
CUDA available: True
GPU: NVIDIA GeForce RTX 4050 Laptop GPU


In [2]:
# Fixed class mapping (used throughout the project)
CLASS_NAMES = [
    "Quartzity",
    "Live_Knot",
    "Marrow",
    "Resin",
    "Dead_Knot",
    "Knot_with_Crack",
    "Knot_missing",
    "Crack",
    "Blue_Stain",
    "Overgrown",
]

CLASS_TO_ID = {name: i for i, name in enumerate(CLASS_NAMES)}
CLASS_TO_ID

{'Quartzity': 0,
 'Live_Knot': 1,
 'Marrow': 2,
 'Resin': 3,
 'Dead_Knot': 4,
 'Knot_with_Crack': 5,
 'Knot_missing': 6,
 'Crack': 7,
 'Blue_Stain': 8,
 'Overgrown': 9}

In [3]:
dataset = load_dataset("iluvvatar/wood_surface_defects", split="train")
print("Total samples:", len(dataset))
dataset[0]

Total samples: 20276


{'id': 160300034,
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2800x1024>,
 'objects': [{'bb': [0.5075, 0.564941, 0.027142, 0.051758],
   'label': 'Dead_Knot'}]}

In [9]:
BASE_DIR = "C:/YOLOv8_OCHD_Project/datasets/wood"
IMG_TRAIN = os.path.join(BASE_DIR, "images", "train")
LBL_TRAIN = os.path.join(BASE_DIR, "labels", "train")
IMG_VAL = os.path.join(BASE_DIR, "images", "val")
LBL_VAL = os.path.join(BASE_DIR, "labels", "val")

for p in [IMG_TRAIN, LBL_TRAIN, IMG_VAL, LBL_VAL]:
    os.makedirs(p, exist_ok=True)

print("Directories ready.")

Directories ready.


In [10]:
for idx, sample in tqdm(enumerate(dataset), total=len(dataset)):
    # Save image
    img_path = os.path.join(IMG_TRAIN, f"{idx}.jpg")
    sample["image"].save(img_path)

    # Save label
    label_path = os.path.join(LBL_TRAIN, f"{idx}.txt")
    with open(label_path, "w") as f:
        for obj in sample["objects"]:
            cls_name = obj["label"]
            if cls_name not in CLASS_TO_ID:
                continue

            cls_id = CLASS_TO_ID[cls_name]
            cx, cy, w, h = obj["bb"]  # YOLO normalized

            f.write(f"{cls_id} {cx} {cy} {w} {h}\n")

print("YOLO-format dataset created.")

100%|████████████████████████████████████████████████████████████████████████████| 20276/20276 [07:28<00:00, 45.24it/s]

YOLO-format dataset created.


In [11]:
all_images = [f for f in os.listdir(IMG_TRAIN) if f.endswith(".jpg")]
random.shuffle(all_images)

split_idx = int(0.8 * len(all_images))
train_files = all_images[:split_idx]
val_files = all_images[split_idx:]

def move(files, img_dst, lbl_dst):
    for f in files:
        os.replace(
            os.path.join(IMG_TRAIN, f),
            os.path.join(img_dst, f)
        )
        os.replace(
            os.path.join(LBL_TRAIN, f.replace(".jpg", ".txt")),
            os.path.join(lbl_dst, f.replace(".jpg", ".txt"))
        )

move(train_files, IMG_TRAIN, LBL_TRAIN)
move(val_files, IMG_VAL, LBL_VAL)

print("Train images:", len(train_files))
print("Val images:", len(val_files))

Train images: 16220
Val images: 4056


In [14]:
sample_imgs = random.sample(os.listdir(IMG_TRAIN), 3)

for img_name in sample_imgs:
    img = cv2.imread(os.path.join(IMG_TRAIN, img_name))
    h, w, _ = img.shape

    with open(os.path.join(LBL_TRAIN, img_name.replace(".jpg", ".txt"))) as f:
        for line in f:
            cls, cx, cy, bw, bh = map(float, line.split())
            cx, cy = int(cx * w), int(cy * h)
            bw, bh = int(bw * w), int(bh * h)

            x1 = int(cx - bw / 2)
            y1 = int(cy - bh / 2)
            x2 = int(cx + bw / 2)
            y2 = int(cy + bh / 2)

            cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), 2)
            cv2.putText(img, CLASS_NAMES[int(cls)], (x1, y1-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)

    cv2.imshow("Sample", img)
    cv2.waitKey(0)

cv2.destroyAllWindows()